In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
tmax = 500
tmin = 10

In [ ]:
full_df = pd.read_csv('./data/ls8_2019_v2_wgs84_merged_climate_extracted.csv')
# full_df = pd.read_csv('./data/ls5_14809735246949.708984_v2_wgs84_merged_climate_extracted.csv')

In [ ]:
# Temporary: Convert to lat/lon
def get_lat_lon(temp_df): 
    temp_df = temp_df[['center_lat', 'center_lon']]
    gdf = gpd.GeoDataFrame(
        temp_df, geometry=gpd.points_from_xy(temp_df.center_lon, temp_df.center_lat),
        crs='ESRI:102033'
    )
    gdf_wgs84 = gdf.to_crs('EPSG:4326')
    
    return gdf_wgs84.geometry.x, gdf_wgs84.geometry.y


In [ ]:
full_df['longitude'], full_df['latitude'] = get_lat_lon(full_df)

In [ ]:
full_df = full_df.loc[full_df['hydropoly_max']<100]
full_df['area_ha'] = full_df['area']*100/10000 # HA
full_df['area_km'] = full_df['area']*100/(1000*1000) # km2
full_df = full_df.loc[full_df['area_ha']<100] # Remove less than 100 ha
full_df['area_m'] = full_df['area']*100


|Variable|Variable name      |Units     |Offset|Scale      |
|--------|-------------------|----------|------|-----------|
|pr      |Precipitation      |mm        |225   |0.006866665|
|Eto     |evapotranspiration |mm        |0     |0.051181102|
|Tmax    |maximum temperature|C         |15    |0.001068148|
|Tmin    |minimum temperature|C         |15    |0.001068148|
|RH      |Relative humidity  |Percentage|0     |0.393700787|
|RS      |Solar radiation    |MJ/m2     |0     |0.157086614|
|U2      |Wind speed         |m/s       |0     |0.059055118|

alpha is reflection coefficient or albedo. A typical
value of the albedo for a grass cover is a = 0.23, whereas
for open water surfaces a = 0.08 (Shuttleworth, 1993; Allen
et al., 1998).

# Method 1: Multiply by evapotranspiration
In the 2019 paper, we just multiplied ETo by area, but I think there should be a scale factor applied (0.6-0.8 maybe) to get lake evaporation from reference Eto.

Actually, not as sure based on this: https://ag.arizona.edu/azmet/et1.htm
"     ETo is a Reference Evapotranspiration.
              ---------

     This is NOT equal to a Pan Evaporation value.

     ETo values can be considered equal to evaporation from a large body 
     of water, such as a pond or lake.  However, for smaller, shallower 
     bodies of water this relationship does not apply.
     Many factors affect lake/pond evaporation including surface area,
     depth, water temperature, and turbidity.
     To date, not enough research has been done to determine the
     minimum body of water size at which ETo is still equal."
     
     
Ok but now this distinguishes betweeen reference ET (a crop) and potential ET (example given as "such as a lake"): https://www.usgs.gov/centers/cfwsc/science/reference-and-potential-evapotranspiration

Another possible ET dataset: https://www.sciencebase.gov/catalog/item/6515cb7ad34e469cabfcdccd
https://www.nature.com/articles/s41597-023-02648-4#Sec6

In [ ]:
evap_scale = 1.0

In [ ]:
# Equation:
# scaling_factor * ETo/1000 = evapotranspiration in m
# Multiply by area_m to get evapotranspiration in m^3
# Multiply by evaporation scaling factor (maybe 0.6 - 0.8, need to find reference) to get evaporation of lake
# Multiply by 1000 to get 
# Multiply by 365 to get annual number? ETo is provided as daily average.
evap_value = ((full_df['et']/1000)*full_df['area_m']*0.051181102).sum()*evap_scale*1000*365

In [ ]:
print(evap_value)

In [ ]:
# Scientific notation
"{:.2E}".format(evap_value)

In [ ]:
evap_value_1984 = 4809735246949.708

In [ ]:
evap_value_2019 = 10701271544365.17

In [ ]:
evap_value_2019/evap_value_1984

In [ ]:
lat_2014_evap = 7.31E11
lat_2014_impoundment_area_ha = 51000


In [ ]:
full_df['area_ha'].sum()/51000

In [ ]:
evap_value/(7.31E11)

In [ ]:
per_capita_daily_use_liters = 841
total_annual_use_liters =(63500000000)*1000

In [ ]:
evap_value/(total_annual_use_liters) # Equivalent to 10% of Brazil's total annual water use? Seems too high

In [ ]:
lat_2014_evap/total_annual_use_liters # Just in Mato Grosso says evaporation is 1%, also seems too high

# Method 2: Use modified Penman Equations

See Valiantzas papers, mainly the first on ewhich ahs an open water calculation. Problem is that that needs to be monthly. Using the new one that is not monthly to start out.1



In [ ]:
Rs = full_df['rs']*0.157086614
tmax_scaled = full_df['tmax']*0.001068148 + 15
tmin_scaled = full_df['tmin']*0.001068148 + 15
Tmean = (tmax_scaled + tmin_scaled)/2
RH = full_df['rh']*0.393700787
u2 = full_df['u2']*0.059055118
lat_radians = np.radians(full_df['latitude']+90)

In [ ]:
vali_et_o = (0.0393*Rs*np.sqrt(Tmean + 9.5)
        - 0.19*(Rs**0.6)*(lat_radians**0.15)
        + 0.048*(Tmean+20)*(1-(RH/100))*(u2**0.7)
       ).

In [ ]:
plt.scatter(vali_et_o, full_df['et']*0.051181102)
plt.plot([2, 6], [2, 6], 'k-', lw=2)

### Method 2B: Adapting simplified equations to open water

Replace alpha (albedo) with open water one (0.08 instead of 0.23)

Replace 0.048 factor with 0.052 because of correction (se equations 21 and 25 in Valiantzas paper)

Replace wind simplification with full factor, but may  not be needed. a_u - 0.38 * 0.54u 

a_u is 1 for full Penman wind factor, 0.5 for reduced. 
But in another place this is just listed as 0.5 + 0.54u

In [ ]:
alpha = 0.08
wind_factor = 1 

In [ ]:
vali_e_open = (
    0.051*(1-alpha)*Rs*np.sqrt(Tmean + 9.5)
    - 0.19*(Rs**0.6)*(lat_radians**0.15)
    + 0.052*(Tmean+20)*(1-(RH/100))*(u2**0.7)
)

In [ ]:
# vali_e_open = (
#     0.051*(1-alpha)*Rs*np.sqrt(Tmean + 9.5)
#     - 0.19*(Rs**0.6)*(lat_radians**0.15)
#     + 0.052*(Tmean+20)*(1-(RH/100))*(wind_factor - 0.38 + 0.54*u2)
# )

In [ ]:
evap_value_vali_e_open = (
    ((vali_e_open/1000)*full_df['area_m']).sum()
    * evap_scale * 1000 * 365)

In [ ]:
evap_value_vali_e_open

In [ ]:
1169665.00*1000000

In [ ]:
1.1E12

In [ ]:
1169665000000

In [ ]:
# Scientific notation
"{:.2E}".format(evap_value_vali_e_open)

In [ ]:
evap_value/0.85

# Sanity check: Lake Mead and Powell combined evaporation: 

Surface area: https://en.wikipedia.org/wiki/Lake_Powell and https://en.wikipedia.org/wiki/Lake_Mead
Evaporation: https://coloradoriverscience.org/Reservoir_evaporation

In [ ]:
mead_powell = 1400001885616 # In liters

In [ ]:
mead_powell_area_ha = 65310 + 64000

In [ ]:
mead_powell/mead_powell_area_ha

In [ ]:
evap_value_2019/full_df['area_ha'].sum()

# Comparisons: 


In [ ]:
acre_feet_to_liters = 1233481.8375

In [ ]:
# US ag irrigation: 
# https://www.nass.usda.gov/Publications/Highlights/2019/2017Census_Irrigation_and_WaterManagement.pdf
us_ag_irrigation = 83400000*acre_feet_to_liters # 83.4 million acre-feet

In [ ]:
"{:.2E}".format(us_ag_irrigation)

In [ ]:
evap_value_vali_e_open/us_ag_irrigation